In [2]:
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

Installed Packages MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0

In [3]:
open System
open MathNet.Numerics
open MathNet.Numerics.Distributions
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Integration


In [4]:
    ///3 point gauss-hermite (https://en.wikipedia.org/wiki/Gauss%E2%80%93Hermite_quadrature)
    ///constants taken from numpy.polynomial.hermite.hermgauss(3) wiht precision 10.
    ///for low point integration to work well, the function should be smooth.
    let gh3 f = 
        let h3 = [1.2247448714;  0.        ;1.2247448714]
        let w3 = [0.2954089752; 1.1816359006 ; 0.2954089752]
        let cons = 1.0/sqrt(System.Math.PI)
        let s2 = sqrt 2.0
        // degree 3
        ((w3,h3) ||> List.map2( fun x y -> x * (f (y*s2))) |> List.sum ) * cons

In [5]:
    let gh34 f = 
        let cons = 1.0/sqrt(System.Math.PI)
        let s2 = sqrt 2.0
        let h3 = [1.2247448714;  0.        ;1.2247448714]
        let w3 = [0.2954089752; 1.1816359006 ; 0.2954089752]
        let d = List.zip h3 w3
        [ for x in d do
            for y in d do
                for z in d do
                    for z1 in d do
                        yield (x,y,z,z1)]
        |> List.map( fun ((x1,y1),(x2,y2),(x3,y3),(x4,y4)) -> 
            let w = y1*y2*y3*y4 
            w * (f (x1*s2) (x2*s2) (x3*s2) (x4*s2))) 
        |> List.sum 
        |> (*) cons

In [6]:
    let appendVector (v1:Vector<float>) (v2:Vector<float>) =
        v1.ToColumnMatrix().Stack(v2.ToColumnMatrix()).Column(0);

    let householderR (q:Vector<float>) =
        let n = q.Count
        let e = DenseVector.init n (fun n -> if n = 0 then 1.0 else 0.0)
        let q' = (q - e )
        let v = q' / q'.L2Norm()
        DiagonalMatrix.identity n - 2.0 * v.OuterProduct(v)

In [7]:

    // a time matrix for VCV using min ( T1 T2 )
    let getTmatrix (T1:Vector<float>) (T2:Vector<float>) =
        let onesT1 = Vector.Build.Dense (T1.Count, 1.)
        let onesT2 = Vector.Build.Dense( T2.Count , 1. )
        let tmatrix2 = onesT1.OuterProduct T2
        let tmatrix1 = T1.OuterProduct onesT2
        tmatrix1.PointwiseMinimum tmatrix2

In [10]:
    ///Sigma = rho v_k v_j min(t_k, t_j)
    let getSigma (v1:Vector<float>) (t1:Vector<float>) (v2:Vector<float>) (t2:Vector<float>) (rho:float) = 
        let tmatrix = getTmatrix t1 t2
        (v1.OuterProduct v2).*tmatrix*rho

In [11]:
    ///get V with Choi's method for 2 assets with constant correlation.
    let getVChoi (f1:Vector<float>) (fw1:Vector<float>) (t1:Vector<float>) (v1:Vector<float>) 
        (f2:Vector<float>) (fw2:Vector<float>) (t2:Vector<float>) v2 (rho:float) = 
            let f1w = f1 .* fw1 //1st asset weighted
            let f2w = f2 .* fw2
            let w = appendVector fw1 fw2 
            let v = appendVector v1 v2 
            let n = f1.Count + f2.Count
            //assuming constant correlation between 2 assets.
            let g' = appendVector f1w f2w 
            let g = (g'/ (g'.L2Norm()))
            let sigma12 = getSigma v1 t1 v2 t2 rho
            let sigma11 = getSigma v1 t1 v1 t1 1.0
            let sigma22 = getSigma v2 t2 v2 t2 1.0
            let sigma = sigma11.Stack( sigma12 ).Append((sigma12.Transpose().Stack(sigma22)))
            let c = sigma.Cholesky().Factor //cholesky
            let den = (sqrt( g.ToRowMatrix() * sigma * g.ToColumnMatrix())).Item(0,0) //should be a scalar 
            let Q1 = (c.Transpose() * g.ToColumnMatrix() ) /den
            let V1 = (c * Q1)// need to adjust so that w_k V_k1 > 0
            let eps = 0.01 //how to choose this? 
            let V1' = DenseVector.init n ( fun n -> 
                if V1.[n,0] * w.[n] > 0. 
                then V1.[n,0] 
                else
                   let s = if w.[n] > 0. then 1.0 else -1.0
                   eps * s * v.[n] )       
            //let mu = 0.99 //how to choose this?
            let Q1' = c.Inverse() * ( V1' / V1'.L2Norm())
            let R = householderR Q1'
            let CR = c * R 
            let CR' = CR.SubMatrix(0, n, 1, n-1) //drop 1st column.
            let svd = CR'.Svd()
            V1* svd.U * svd.W